In [1]:
import time
import openai
import pandas as pd
import glob
from tqdm import tqdm
import os
import dotenv
import requests
import json
import csv

In [2]:
#df = pd.read_csv('./dataset/test.csv')

#random_sample = df.sample(n=100, random_state=42)

#random_sample.to_csv('./dataset/gpt_test.csv', index=False)

random_sample = pd.read_csv('./dataset/gpt_test.csv')

text_list = random_sample['lyrics'].tolist()

lyrics = text_list

In [10]:
def dividir_por_tokens(lista):
    grupos = []
    grupo_atual = []
    contador_tokens = 0

    for string in lista:
        tokens = len(string)

        if contador_tokens + tokens <= 900:
            grupo_atual.append(string)
            contador_tokens += tokens
        else:
            grupos.append(grupo_atual)
            grupo_atual = [string]
            contador_tokens = tokens

    if grupo_atual: 
        grupos.append(grupo_atual)
    
    return grupos

In [ ]:
#Zero-shot comand:
#Analyze the following lyrics and determine if their sentiment is either 'positive', 'negative' or 'neutral'. Format your answer as a CSV file with a column for the sentiment category and another for the lyrics. Lyrics to analyze: {texto}"

#One-shot command:
#"The following lyrics is POSITIVE: 'goodness sake got hippy hippy shake got shake got hippy hippy shake ooh cant sit still hippy hippy shake get fill hippy hippy shake yeah bag ooh hippy hippy shake well shake left shake right hippy shake shake might ooh baby ooh yeah come shake ooh bag ohthe hippy hippy shake well shake left shake right hippy shake shake might oh baby yeah come shake oh bag ooh hippy hippy shake ooh hippy hippy shake ooh hippy hippy shake'. \
#The following lyrics is NEGATIVE: 'long year since youve gone ive alone ive grown old fall piece im falling fell piece im still falling every time im falling alone fall piece keep journal memory im feeling lonely cant breathe fall piece im falling fell piece im still falling every time im falling alone fall piece every time im falling alone fall piece every time im falling alone fall piece year ive tried go memory die im waiting find find falling im falling every time im falling alone fall piece every time im falling alone fall piece every time im falling alone fall piece every time im falling alone fall piece'. \
# The following lyrics is NEUTRAL: 'say demon say demon rock im rock gibraltar say demon say think know im getting kind low think know im getting kind low made sun shine take air breathe let back tell make think know im getting kind low think know im getting kind low wanted dont say'. \
#Analyze the following lyrics and determine if their sentiment is either POSITIVE, NEGATIVE or NEUTRAL: '{texto}'"

#Few-shot command:
#The following lyrics is POSITIVE: 'goodness sake got hippy hippy shake got shake got hippy hippy shake ooh cant sit still hippy hippy shake get fill hippy hippy shake yeah bag ooh hippy hippy shake well shake left shake right hippy shake shake might ooh baby ooh yeah come shake ooh bag ohthe hippy hippy shake well shake left shake right hippy shake shake might oh baby yeah come shake oh bag ooh hippy hippy shake ooh hippy hippy shake ooh hippy hippy shake'. \
#The following lyrics is NEGATIVE: 'long year since youve gone ive alone ive grown old fall piece im falling fell piece im still falling every time im falling alone fall piece keep journal memory im feeling lonely cant breathe fall piece im falling fell piece im still falling every time im falling alone fall piece every time im falling alone fall piece every time im falling alone fall piece year ive tried go memory die im waiting find find falling im falling every time im falling alone fall piece every time im falling alone fall piece every time im falling alone fall piece every time im falling alone fall piece'. \
#The following lyrics is NEUTRAL: 'say demon say demon rock im rock gibraltar say demon say think know im getting kind low think know im getting kind low made sun shine take air breathe let back tell make think know im getting kind low think know im getting kind low wanted dont say'. \
#The following lyrics is NEGATIVE: 'voice aeon angelus satani os pro nobis lucifer alone suffered fall torment shame ill smite heaven golden pride never pity thee immaculate divine satan elohim none dare stand way thou bow none eden feculence conjure serpent messenger savior world decay concord temptation fall eve thine kingdom power thine kingdom glory forever scholar unlight great volcano excrement rippling eager erupt reconstellate firmament behold great accuser megalinga throbbing zeal raptor yearning pierce rape seventh seal destroyer cosmos implore ungod implode sun there none wronging serpent cult untouched ignored serpent might trumpet blow shadow horn thine kingdom power thine kingdom glory forever lightning stab heaven thou quiver mount sinai spit rock tablet cofenant listen tempest rasp azazel apolyon black sun never set never rose habemus satanas caesar wrath empire fall petition lord fire santa sede burn phallus dei erect pole pole viva blasfemia coitus infinitus hear universe scream bleeding black hole horn careless god mourns'. \
#The following lyrics is POSITIVE: 'slowly turn page see smiling face hard believe today ten year ago invitation wedding celebration look promise picture show know loved book would right side fool overlooked promise love forever worse better wed still together idve loved book opened book often wouldnt forgotten much cared much meant solemn vow honor hold read know late say im sorry loved book would right side fool overlooked promise love forever worse better wed still together idve loved book loved book'. \
#The following lyrics is NEUTRAL: 'staring window tree cant see horizon see shes moment im knee ophelia standing doorway heart cannot together apart shes caught line denies every time ophelia let take want go pillar soul one know conversation halt sputter like flame falter reappears im caught word say name ophelia led forest lark content leave stumbling dark ive known time one false move mine ophelia let take want go pillar soul one know ophelia mustnt try tell feel cause everything dream real cant let go ophelia one else tell feel cause everything dream real dont ever let go' \
#Analyze the following lyrics and determine if their sentiment is either POSITIVE, NEGATIVE or NEUTRAL: '{texto}'"

In [41]:
dotenv.load_dotenv()

def analise_sentimento(texto: str, i: int) -> str:
    CHAVE_API = ""
    modelo_engine = "text-davinci-003"
    
    
    
    comando = f"The following lyrics is POSITIVE: 'goodness sake got hippy hippy shake got shake got hippy hippy shake ooh cant sit still hippy hippy shake get fill hippy hippy shake yeah bag ooh hippy hippy shake well shake left shake right hippy shake shake might ooh baby ooh yeah come shake ooh bag ohthe hippy hippy shake well shake left shake right hippy shake shake might oh baby yeah come shake oh bag ooh hippy hippy shake ooh hippy hippy shake ooh hippy hippy shake'. \
               The following lyrics is NEGATIVE: 'long year since youve gone ive alone ive grown old fall piece im falling fell piece im still falling every time im falling alone fall piece keep journal memory im feeling lonely cant breathe fall piece im falling fell piece im still falling every time im falling alone fall piece every time im falling alone fall piece every time im falling alone fall piece year ive tried go memory die im waiting find find falling im falling every time im falling alone fall piece every time im falling alone fall piece every time im falling alone fall piece every time im falling alone fall piece'. \
               The following lyrics is NEUTRAL: 'say demon say demon rock im rock gibraltar say demon say think know im getting kind low think know im getting kind low made sun shine take air breathe let back tell make think know im getting kind low think know im getting kind low wanted dont say'. \
               The following lyrics is NEGATIVE: 'voice aeon angelus satani os pro nobis lucifer alone suffered fall torment shame ill smite heaven golden pride never pity thee immaculate divine satan elohim none dare stand way thou bow none eden feculence conjure serpent messenger savior world decay concord temptation fall eve thine kingdom power thine kingdom glory forever scholar unlight great volcano excrement rippling eager erupt reconstellate firmament behold great accuser megalinga throbbing zeal raptor yearning pierce rape seventh seal destroyer cosmos implore ungod implode sun there none wronging serpent cult untouched ignored serpent might trumpet blow shadow horn thine kingdom power thine kingdom glory forever lightning stab heaven thou quiver mount sinai spit rock tablet cofenant listen tempest rasp azazel apolyon black sun never set never rose habemus satanas caesar wrath empire fall petition lord fire santa sede burn phallus dei erect pole pole viva blasfemia coitus infinitus hear universe scream bleeding black hole horn careless god mourns'. \
               The following lyrics is POSITIVE: 'slowly turn page see smiling face hard believe today ten year ago invitation wedding celebration look promise picture show know loved book would right side fool overlooked promise love forever worse better wed still together idve loved book opened book often wouldnt forgotten much cared much meant solemn vow honor hold read know late say im sorry loved book would right side fool overlooked promise love forever worse better wed still together idve loved book loved book'. \
               The following lyrics is NEUTRAL: 'staring window tree cant see horizon see shes moment im knee ophelia standing doorway heart cannot together apart shes caught line denies every time ophelia let take want go pillar soul one know conversation halt sputter like flame falter reappears im caught word say name ophelia led forest lark content leave stumbling dark ive known time one false move mine ophelia let take want go pillar soul one know ophelia mustnt try tell feel cause everything dream real cant let go ophelia one else tell feel cause everything dream real dont ever let go' \
               Analyze the following lyrics and determine if their sentiment is either POSITIVE, NEGATIVE or NEUTRAL: '{texto}'"
    
    cabecalho = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {CHAVE_API}"
    }

    dados = {
        "prompt": comando,
        "temperature": 0,
        "max_tokens": 2047,
        "n": 1,
        "stop": None,
    }

    resposta = requests.post(
        f"https://api.openai.com/v1/engines/{modelo_engine}/completions",
        headers=cabecalho,
        json=dados
    )

    resposta_json = resposta.json()
    resultado = resposta_json['choices'][0]['text'].strip()

    data = {
        'resultado': resultado,
    }

    return data

In [42]:
grupos = dividir_por_tokens(lyrics)
few_shot = []

for i, grupo in enumerate(grupos):
    resultado = analise_sentimento(grupo, i)
    print(resultado)
    few_shot.append(resultado)

{'resultado': "The following lyrics is POSITIVE: 'day window watch pass say youre lucky girl boy like truly dream come true girl world belongs imagination runnin away imagination runnin away soon well married raise family cozy little home country two child maybe three tell visualize couldnt dream real seems every night knee pray dear lord hear plea dont ever let another take love surely die love heavenly arm enfold hear tender rhapsody reality doesnt even know name oh shame imagination runnin away runnin runnin imagination windmill ind would think time imagination didnt even know name oh shame well imagination'."}
{'resultado': 'The sentiment of the lyrics is POSITIVE.'}
{'resultado': 'The sentiment of the lyrics is POSITIVE.'}
{'resultado': 'The sentiment of the lyrics is NEGATIVE.'}
{'resultado': 'The sentiment of the lyrics is NEUTRAL.'}
{'resultado': 'The sentiment of the lyrics is POSITIVE.'}
{'resultado': 'The sentiment of the lyrics is POSITIVE.'}
{'resultado': 'The sentiment of

In [36]:
one_shot

[{'resultado': "The following lyrics is POSITIVE: 'day window watch pass say youre lucky girl boy like truly dream come true girl world belongs imagination runnin away imagination runnin away soon well married raise family cozy little home country two child maybe three tell visualize couldnt dream real seems every night knee pray dear lord hear plea dont ever let another take love surely die love heavenly arm enfold hear tender rhapsody reality doesnt even know name oh shame imagination runnin away runnin runnin imagination windmill ind would think time imagination didnt even know name oh shame well imagination'."},
 {'resultado': 'The sentiment of the lyrics is POSITIVE, as it speaks of hope and survival.'},
 {'resultado': 'The sentiment of the lyrics is POSITIVE.'},
 {'resultado': 'The sentiment of the lyrics is NEGATIVE.'},
 {'resultado': 'The sentiment of the lyrics is NEUTRAL.'},
 {'resultado': 'The sentiment of the lyrics is POSITIVE.'},
 {'resultado': 'The sentiment of the lyric

In [43]:
few_shot

[{'resultado': "The following lyrics is POSITIVE: 'day window watch pass say youre lucky girl boy like truly dream come true girl world belongs imagination runnin away imagination runnin away soon well married raise family cozy little home country two child maybe three tell visualize couldnt dream real seems every night knee pray dear lord hear plea dont ever let another take love surely die love heavenly arm enfold hear tender rhapsody reality doesnt even know name oh shame imagination runnin away runnin runnin imagination windmill ind would think time imagination didnt even know name oh shame well imagination'."},
 {'resultado': 'The sentiment of the lyrics is POSITIVE.'},
 {'resultado': 'The sentiment of the lyrics is POSITIVE.'},
 {'resultado': 'The sentiment of the lyrics is NEGATIVE.'},
 {'resultado': 'The sentiment of the lyrics is NEUTRAL.'},
 {'resultado': 'The sentiment of the lyrics is POSITIVE.'},
 {'resultado': 'The sentiment of the lyrics is POSITIVE.'},
 {'resultado': 'T

In [48]:
# Nome do arquivo JSON para salvar os resultados
nome_arquivo_json = "few-shot.json"

# Salvar os resultados no arquivo JSON
with open(nome_arquivo_json, 'w') as arquivo_json:
    json.dump(few_shot, arquivo_json)

print("Dados salvos com sucesso no arquivo JSON.")



Dados salvos com sucesso no arquivo JSON.
